In [1]:
import numpy as np
from pydrake.all import (
    DiagramBuilder,
    AddMultibodyPlantSceneGraph,
    Parser,
    MeshcatVisualizer,
    StartMeshcat,
    MeshcatVisualizerParams,
    Simulator,
    JointSliders,
    InverseKinematics,
    RotationMatrix,
    Solve,
    ContactVisualizerParams,
    ContactVisualizer,
    GeometrySet,
    CollisionFilterDeclaration,
    Role,
    eq,
    RigidTransform,
)
from manipulation import ConfigureParser, running_as_notebook
import pydot
from IPython.display import display, Image, SVG

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://99019c02-9e81-448d-8532-b07e66cf3903.deepnoteproject.com/7000/
Installing NginX server for MeshCat on Deepnote...


## Problem Description
We will solve inverse kinematics for one arm on a PR2 robot. This robot model has been modified to have convex collision geometries, and to simplify this problem, we have welded (fixed) several joints that are irrelavent to the kinematics of the arms.

In [3]:
def build_env():
    """Load in models and build the diagram."""
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.AddModelsFromUrl("package://manipulation/pr2_shelves.dmd.yaml")
    plant.Finalize()

    MeshcatVisualizer.AddToBuilder(
        builder,
        scene_graph.get_query_output_port(),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False),
    )

    diagram = builder.Build()
    return diagram, plant, scene_graph

In [4]:
def filterCollsionGeometry(scene_graph, context=None):
    """Some robot models may appear to have self collisions due to overlapping collision geometries.
    This function filters out such problems for our PR2 model."""
    if context is None:
        filter_manager = scene_graph.collision_filter_manager()
    else:
        filter_manager = scene_graph.collision_filter_manager(context)
    inspector = scene_graph.model_inspector()

    pr2 = {}
    shelves = []
    tables = []

    for gid in inspector.GetGeometryIds(
        GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity
    ):
        gid_name = inspector.GetName(inspector.GetFrameId(gid))
        if "pr2" in gid_name:
            link_name = gid_name.split("::")[1]
            pr2[link_name] = [gid]

    def add_exclusion(set1, set2=None):
        if set2 is None:
            filter_manager.Apply(
                CollisionFilterDeclaration().ExcludeWithin(GeometrySet(set1))
            )
        else:
            filter_manager.Apply(
                CollisionFilterDeclaration().ExcludeBetween(
                    GeometrySet(set1), GeometrySet(set2)
                )
            )

    # Robot-to-self collisions
    add_exclusion(
        pr2["base_link"],
        pr2["l_shoulder_pan_link"]
        + pr2["r_shoulder_pan_link"]
        + pr2["l_upper_arm_link"]
        + pr2["r_upper_arm_link"]
        + pr2["head_pan_link"]
        + pr2["head_tilt_link"],
    )
    add_exclusion(
        pr2["torso_lift_link"], pr2["head_pan_link"] + pr2["head_tilt_link"]
    )
    add_exclusion(
        pr2["l_shoulder_pan_link"] + pr2["torso_lift_link"],
        pr2["l_upper_arm_link"],
    )
    add_exclusion(
        pr2["r_shoulder_pan_link"] + pr2["torso_lift_link"],
        pr2["r_upper_arm_link"],
    )
    add_exclusion(pr2["l_forearm_link"], pr2["l_gripper_palm_link"])
    add_exclusion(pr2["r_forearm_link"], pr2["r_gripper_palm_link"])

In [5]:
goal_rotation = RotationMatrix(
    [
        [1, 0, 0],
        [0, -1, 0],
        [0, 0, -1],
    ]
)
goal_position = np.array([-0.83, 0.18, 1.4])
goal_pose = RigidTransform(goal_rotation, goal_position)

In [6]:
def AddOrientationConstraint(ik, R_WG, bounds):
        """Add orientation constraint to the ik problem. Implements an inequality
        constraint where the axis-angle difference between f_R(q) and R_WG must be
        within bounds. Can be translated to:
        ik.prog().AddBoundingBoxConstraint(angle_diff(f_R(q), R_WG), -bounds, bounds)
        """
        ik.AddOrientationConstraint(
            frameAbar=world_frame,
            R_AbarA=R_WG,
            frameBbar=gripper_frame,
            R_BbarB=RotationMatrix(),
            theta_bound=bounds,
        )

def AddPositionConstraint(ik, p_WG_lower, p_WG_upper):
    """Add position constraint to the ik problem. Implements an inequality
    constraint where f_p(q) must lie between p_WG_lower and p_WG_upper. Can be
    translated to
    ik.prog().AddBoundingBoxConstraint(f_p(q), p_WG_lower, p_WG_upper)
    """
    ik.AddPositionConstraint(
        frameA=world_frame,
        frameB=gripper_frame,
        p_BQ=np.zeros(3),
        p_AQ_lower=p_WG_lower,
        p_AQ_upper=p_WG_upper,
    )


### IK for a Mobile Manipulator

Given a RigidTransform X_WG, compute a robot configuration placing the left gripper at that pose. We use optimization to solve the IK problem, and we repeatedly solve the program with random initializations until it succeeds. We have implemented a skeleton of the necessary code in the following function, but you must complete several pieces:

- Add position and orientation constraints to the gripper frame. The end effector should match the desired pose to within 1mm translation along each axis, and the rotation should be off by no more than 1 degree.
- If `fix_base` is True, constrain the base pose $(x,y,\theta)$ to be equal to `base_pose`.
- Add a collision free constraint with [AddMinimumDistanceLowerBoundConstraint](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_inverse_kinematics.html#a2ecd71efd675a7e1a4293adb05c9b9df). The minimum distance between any pair of collision geometries should be at least 1cm.
- Compute a random initial guess for the joint angles within the robot's joint limits. You can access the joint limits from the multibody plant, but some of the joints are angle-valued and don't have limits. For these joints, use the range $[-\pi,\pi]$.

In [7]:
import math

In [8]:
def solve_ik(X_WG, max_tries=10, fix_base=False, base_pose=np.zeros(3)):
    diagram, plant, scene_graph = build_env()

    world_frame = plant.world_frame()
    gripper_frame = plant.GetFrameByName("l_gripper_palm_link")

    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    sg_context = scene_graph.GetMyContextFromRoot(context)
    filterCollsionGeometry(scene_graph, sg_context)

    # Note: passing in a plant_context is necessary for collision-free constraints!
    ik = InverseKinematics(plant, plant_context)
    q_variables = ik.q()  # Get variables for MathematicalProgram
    prog = ik.prog()  # Get MathematicalProgram
    q_nominal = np.zeros(len(q_variables))
    q_nominal[0:3] = base_pose
    prog.AddQuadraticErrorCost(
        np.eye(len(q_variables)), q_nominal, q_variables
    )

    def AddOrientationConstraint(ik, R_WG, bounds):
        """Add orientation constraint to the ik problem. Implements an inequality
        constraint where the axis-angle difference between f_R(q) and R_WG must be
        within bounds. Can be translated to:
        ik.prog().AddBoundingBoxConstraint(angle_diff(f_R(q), R_WG), -bounds, bounds)
        """
        ik.AddOrientationConstraint(
            frameAbar=world_frame,
            R_AbarA=R_WG,
            frameBbar=gripper_frame,
            R_BbarB=RotationMatrix(),
            theta_bound=bounds,
        )

    def AddPositionConstraint(ik, p_WG_lower, p_WG_upper):
        """Add position constraint to the ik problem. Implements an inequality
        constraint where f_p(q) must lie between p_WG_lower and p_WG_upper. Can be
        translated to
        ik.prog().AddBoundingBoxConstraint(f_p(q), p_WG_lower, p_WG_upper)
        """
        ik.AddPositionConstraint(
            frameA=world_frame,
            frameB=gripper_frame,
            p_BQ=np.zeros(3),
            p_AQ_lower=p_WG_lower,
            p_AQ_upper=p_WG_upper,
        )


    # Add your constraints here
    # pose = plant.GetPositions(plant_context)
    print("pose",  X_WG.translation())
    if fix_base:
        AddPositionConstraint(ik, base_pose, base_pose)
    else:
        AddPositionConstraint(ik, X_WG.translation() - 0.001, X_WG.translation() + 0.001)
    AddOrientationConstraint(ik, X_WG.rotation(), np.pi/180)
    ik.AddMinimumDistanceLowerBoundConstraint(0.01)

    for count in range(max_tries):
        # Compute a random initial guess here
        lower_limit = plant.GetPositionLowerLimits()
        upper_limit = plant.GetPositionUpperLimits()
        q_guess = []
        for l, u in zip(lower_limit, upper_limit):
            if (math.isinf(l) or math.isinf(u)):
                q_guess.append(np.random.uniform(-np.pi, np.pi))
            else:
                q_guess.append(np.random.uniform(l, u))
        print(q_guess)
        prog.SetInitialGuess(q_variables, q_guess)
        

        result = Solve(prog)

        if running_as_notebook:
            render_context = diagram.CreateDefaultContext()
            plant.SetPositions(
                plant.GetMyContextFromRoot(render_context),
                result.GetSolution(q_variables),
            )
            diagram.ForcedPublish(context)

        if result.is_success():
            print("Succeeded in %d tries!" % (count + 1))
            return result.GetSolution(q_variables)

    print("Failed!")
    return None

First, we show an example where we have fixed the base. It may take many tries for it to solve, or not solve at all! At each iteration, we visualize where the optimizer stopped, so you can see what the failures look like.

In [9]:
solve_ik(
    goal_pose,
    max_tries=20,
    fix_base=True,
    base_pose=np.array([-1.23, 0.05, 0]),
)

INFO:drake:PackageMap: Downloading https://github.com/RobotLocomotion/models/archive/90397c33cab9b7234d94c1018f2755bb9989c5a8.tar.gz
pose [-0.83  0.18  1.4 ]
[2.9341805983558693, 2.1987890191183865, -2.576131176851423, 0.07798801256712773, -0.5257911611328887, -0.33928491210397344, -0.9550937980914571, -0.6136214371599653, 0.9310488217049491, -1.2001104423642093, -1.3977142955297375, 1.9394351909352585, 0.656449894610389, -0.6983048299500205, -0.9260800485057059, 1.2915456555247022, -0.7252548813309125, 1.2577319862013558]
[-0.3836697000566889, 0.9915758204732716, -0.27677072819954773, 0.039902351348394635, -1.5752585668266486, 0.808764664351498, -1.4186956900243204, -1.482079188706388, -0.9539942197130822, -1.4500554202903504, 1.6829436015298045, 2.02787421092264, 0.5367802047319511, 1.4155805245668411, -0.6532251318108313, 0.40659199030855797, -1.3315657199129105, -1.6936814670769487]
[-1.501910781274463, -2.87596158719406, 2.366760551973986, 0.032907921278608504, -1.8957225718266062

When we allow the base to move freely, the inverse kinematics can be solved much more easily!

In [12]:
solve_ik(goal_pose, fix_base=False)

pose [-0.83  0.18  1.4 ]
[0.7186403536942265, -0.5799111890001214, 1.2281186163209465, 0.19846679456309402, -1.1847402532952576, 0.18206114232427983, -2.259699393416021, -0.9626902368933414, 0.6087424491588713, -0.9547524486160555, 1.9890331364647187, -0.708469462404397, -0.2645227714866687, 3.4680738083578984, -0.9942593622814055, -0.9397475910969444, -1.4140363884218474, -1.3238397841481153]
Succeeded in 1 tries!


array([-1.22794258e+00, -4.14254191e-01,  5.37910491e-01,  2.37085137e-01,
        2.65581847e-01,  4.10503127e-01, -4.35881849e-04,  0.00000000e+00,
        1.05355984e-02,  0.00000000e+00,  2.31264708e-07,  2.91817235e-01,
       -4.94832144e-01,  1.13280900e+00, -3.03622801e-01,  1.05273971e+00,
       -7.48296225e-01,  6.11619021e-01])

In [11]:
from manipulation.exercises.mobile.test_mobile_base_ik import (
    TestMobileBaseIk,
)
from manipulation.exercises.grader import Grader

Grader.grade_output([TestMobileBaseIk], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 5/5.

Score for Test solve_ik is 5/5.
- pose [-0.83  0.18  1.4 ]
[-1.3321961097564856, 1.1940287432204073, 2.010193316579689, 0.00438110404943313, -0.9632988857843279, 0.3687191625526397, -3.239782697049808, -2.110310799494221, 1.8137295424641398, -1.4540297280040306, 1.221850051616732, -0.4309314982696136, 1.2059200872542641, 0.44779159415996883, -0.8382526914435626, 0.520431970594335, -0.655371883929335, 1.153224898456643]
Succeeded in 1 tries!
pose [0.83 0.18 1.1 ]
[0.18362175545714532, 2.5141997292379274, 2.4791519845069008, 0.1938974049230109, -1.824040328223564, 1.0401584009621296, -3.430207601425807, -1.5276485622763856, -0.03984505359355239, -0.4937388765005377, -2.887493079434199, 0.5719171157930523, -0.24324684063323315, 3.156743797913559, -0.07543533014735804, -0.02782737887454756, -1.7919848000695238, 2.2311813997756555]
Succeeded in 1 tries!
pose [0.   1.13 1.6 ]
[-2.7692079964595635, 3.1282630089881307, -1.7879138461416093, 0.15876422863654932, -0.318263457

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=99019c02-9e81-448d-8532-b07e66cf3903' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>